In [ ]:
import urllib.request
import json
import os
import ssl
import time

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

url = os.getenv("PROMPTFLOW_ENDPOINT")
api_key = os.getenv("PROMPTFLOW_API_KEY")

if not url:
    raise Exception("The API endpoint URL must be provided in the environment variable 'PROMPTFLOW_ENDPOINT'.")

if not api_key:
    raise Exception("The API key must be provided in the environment variable 'PROMPTFLOW_API_KEY'.")

chat_history = [{"inputs": {"query": "Hi"},"outputs": {"reply": "Hello, what can I help you with today? Anything you want to buy?"}}]
q = "What did John Smith buy?"
data = {
    "chat_history": chat_history,
    "query": q
}

body = str.encode(json.dumps(data))
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + api_key
    #,'azureml-model-deployment': 'ep-prj005-dec05-1'
}

# Debugging:
#print("Request URL:", url)
print("Request Body:", json.dumps(data, indent=2))

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    a = json.loads(result).get("reply", "The requested information is not available in the retrieved data.")
    print(a)
    time.sleep(1)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))